In [1]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Error loading punkt: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>


In [18]:
%pip install google.colab
from google.colab import files
import os

file_name = 'intents.json'
if file_name not in os.listdir():
    uploaded = files.upload()
    print("File 'intents.json' Already Uploaded.")
else:
    os.remove(file_name)
    uploaded = files.upload()
    os.listdir()
    print("File 'intents.json' Already Updated.")


     ---------------------------------------- 0.0/72.9 kB ? eta -:--:--
     ---------------------------------------- 72.9/72.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/11.8 MB 2.2 MB/s eta 0:00:06
     ---------------------------------------- 0.1/11.8 MB 2.1 MB/s eta 0:00:06
      --------------------------------------- 0.2/11.8 MB 1.6 MB/s eta 0:00:08
     - -------------------------------------- 0.5/11.8 MB 2.4 MB/s eta 0:00:05
     -- ------------------------------------- 0.6/11.8 MB 2.7 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/11.8 MB 3.0 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/11.8 MB 3.3 MB/s eta 0:00:04
     --- ------------------------------------ 1.0/11.8 MB 2.7 MB/s eta 0:00:04
     --- ---------------------------------

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [841 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\pandas
      copying pandas\conftest.py -> build\lib.win-amd64-3.9\pandas
      copying pandas\testing.py -> build\lib.win-amd64-3.9\pandas
      copying pandas\_version.py -> build\lib.win-amd64-3.9\pandas
      copying pandas\__init__.py -> build\lib.win-amd64-3.9\pandas
      creating build\lib.win-amd64-3.9\pandas\api
      copying pandas\api\__init__.py -> build\lib.win-amd64-3.9\pandas\api
      creating build\lib.win-amd64-3.9\pandas\arrays
      copying pandas\arrays\__init__.py -> build\lib.win-amd64-3.9\pandas\arrays
      creating build\lib.win-amd64-3.9\pandas\compat
      copying pandas\compat\chainmap.py -> build\lib.win-amd64-3.9\pandas\compat
      copying pandas\

ModuleNotFoundError: No module named 'google.colab'

In [19]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [20]:
intents

{'intents': [{'tag': 'Morning_Greetings',
   'patterns': ['Good morning', 'Morning'],
   'responses': ['Morning! Can CuBot help you?',
    'Good Morning! How was your day?',
    'Morning! How are you doing?',
    'Morning! What are you thinking about?',
    'Good Morning! Have a happy day.',
    'Hi! Good Morning! What can CuBot do for you?',
    'Beautiful morning! How are you today?',
    'Good Morning! Is there anything special you want to do today?',
    'A fresh morning! Anything to discuss?']},
  {'tag': 'Afternoon Greetings',
   'patterns': ['Good afternoon', 'Afternoon'],
   'responses': ['Good Afternoon! Is there something you want to tell me?',
    'Afternoon! How are you doing today?',
    'Good Afternoon! Hope you had a delicious lunch.',
    'Afternoon! How can CuBot help you?',
    "Afternoon! Is there anything you'd like to discuss?",
    'Good Afternoon! How are you planning today?',
    'Hi! Good Afternoon! What can CuBot do for you?',
    'Nice afternoon! How are you 

In [21]:
words = []
classes = []
documents = []
ignore = ['?','!',',']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [22]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

167 documents
24 classes ['Afternoon Greetings', 'Comments', 'Common_response_1', 'Common_response_2', 'CuBot_Ask_for_Comments', 'CuBot_News', 'CuBot_Thank_Comment', 'Default', 'Evening_Greetings', 'Farewell', 'General_Greetings', 'General_Response_10', 'General_Response_11', 'General_Response_3', 'General_Response_4', 'General_Response_5', 'General_Response_6', 'General_Response_7', 'General_Response_8', 'General_Response_9', 'Morning_Greetings', 'Night_Greetings', 'Pattern_not_found', 'Thank_CuBot']
158 unique stemmed words ["'ll", "'m", "'re", "'s", "'ve", '.', 'a', 'accept', 'admir', 'affection', 'afraid', 'afternoon', 'again', 'all', 'alon', 'alreadi', 'am', 'amaz', 'and', 'angri', 'ani', 'annoy', 'anxiou', 'appreci', 'are', 'at', 'awe', 'awesom', 'bad', 'becaus', 'broken', 'bye', 'ca', 'can', 'clueless', 'comment', 'confus', 'continu', 'cool', 'cubot', 'cute', 'day', 'deep', 'difficult', 'disappoint', 'disbelief', 'disgust', 'dissatisfi', 'disturb', 'do', 'empti', 'enchant', 'eve

In [24]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training, dtype='object')

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [25]:
from keras import regularizers

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, input_shape=(len(train_x[0]),),activation='relu'))
model.add(tf.keras.layers.Dense(10, kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\jessi\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.0210 - loss: 3.2998 
Epoch 2/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.0029 - loss: 3.2630   
Epoch 3/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.0507 - loss: 3.2223
Epoch 4/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.0690 - loss: 3.1679
Epoch 5/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.1013 - loss: 3.1129
Epoch 6/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.0837 - loss: 3.0677
Epoch 7/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step - accuracy: 0.1189 - loss: 2.9918
Epoch 8/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.1656 - loss: 2.8971
Epoch 9/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - accuracy: 0.1283 - loss: 2.9058   
Epoch 10/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - accuracy: 0.1609 - loss: 2.7534
Epoch 11/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.2125 - loss: 2.7133
Epoch 12/500
21/21 ━━━━━━━━━━━━━━━

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `tf.saved_model.save()` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=model.pkl.

In [27]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [28]:
from keras.models import load_model
model = load_model("model.pkl")

ValueError: File format not supported: filepath=model.pkl. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(model.pkl, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [29]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [30]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [37]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def save_comment(username, comment):
    folder_name = 'Comments'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    filename = os.path.join(folder_name, f"{username}_comments.txt")
    with open(filename, 'a') as file:
        file.write(comment + '\n')

        

In [35]:
import pyttsx3
import speech_recognition as sr

def assistant_speaks(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()


def get_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        print("Recognizing...")
        user_input = recognizer.recognize_google(audio)
        print(f"You said: {user_input}")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I couldn't understand what you said.")
        return ""
    except sr.RequestError as e:
        print(f"Error: {e}")
        return ""



assistant_speaks("Hello, what's your name?")
user_name = get_audio()
assistant_speaks("Hello, " + user_name)
exited = False
default_found = False

while not default_found:
  user_input = "CuBot"
  results = classify(user_input)
  if results:
    for i in intents['intents']:
        if results[0][0] == i['tag'] == 'Default':
              assistant_speaks(f"{random.choice(i['responses'])}")
              default_found = True
              break

while not exited:
  user_input = get_audio()
  results = classify(user_input)

  if results:
      for i in intents['intents']:
          if results[0][0] == i['tag'] == 'Farewell':
              assistant_speaks(f"{random.choice(i['responses'])}")
              user_input = "Do you have any suggestion?"
              results = classify(user_input)
              for j in intents['intents']:
                if j['tag'] == 'CuBot_Ask_for_Comments':
                    comment = assistant_speaks(f"{random.choice(j['responses'])}")
                    save_comment(user_name, comment)
                    user_input = "I already gave the comment"
                    results = classify(user_input)
                    break
              for k in intents['intents']:
                if k['tag'] == 'CuBot_Thank_Comment':
                    assistant_speaks(f"{random.choice(k['responses'])}")
                    exited = True
                    break
              break
          elif results[0][0] == i['tag'] == 'Pattern_not_found':
              assistant_speaks(f"{random.choice(i['responses'])}")
              break
          elif results[0][0] == i['tag']:
              assistant_speaks(f"{random.choice(i['responses'])}")
              break
  else:
      for i in intents['intents']:
          if i['tag'] == 'Pattern_not_found':
              assistant_speaks(f"{random.choice(i['responses']):<{len(user_name)}}")
              break
          
          


Listening...
Recognizing...
You said: got to be sudden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Listening...
Recognizing...
You said: exit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Listening...
Recognizing...
You said: bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


NameError: name 'os' is not defined